In [56]:
import time
import requests
import random
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options

In [35]:
USER_AGENTS = [
 "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
 "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
 "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
 "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
 "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
 "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
 "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
 "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
 "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
 "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
 "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
 "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
 "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
 "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
 "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
 "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
]

In [41]:
# Reference: https://stackoverflow.com/questions/54245193/i-cant-get-body-when-request-from-bloomberg-url
driver_path = r"./chromedriver.exe"
driver = webdriver.Chrome(driver_path)
url = "https://www.bloomberg.com/search?query=AAPL"
headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9,fr;q=0.8,ro;q=0.7,ru;q=0.6,la;q=0.5,pt;q=0.4,de;q=0.3',
    'cache-control': 'max-age=0',
    'upgrade-insecure-requests': '1',
    'user-agent': '{}'.format(random.choice(USER_AGENTS))
}

# res = requests.get(url, timeout=10, headers=headers)
# soup = BeautifulSoup(res.text, "html.parser")
driver.get(url)
soup = BeautifulSoup(driver.page_source, "html.parser")

In [43]:
def bloomberg_crawler_generator(query="AAPL"):
    # Reference: https://stackoverflow.com/questions/54245193/i-cant-get-body-when-request-from-bloomberg-url
    driver_path = r"./chromedriver.exe"
    driver = webdriver.Chrome(driver_path)
    url = "https://www.bloomberg.com/search?query={}".format(query)
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.9,fr;q=0.8,ro;q=0.7,ru;q=0.6,la;q=0.5,pt;q=0.4,de;q=0.3',
        'cache-control': 'max-age=0',
        'upgrade-insecure-requests': '1',
        'user-agent': '{}'.format(random.choice(USER_AGENTS))
    }

    # res = requests.get(url, timeout=10, headers=headers)
    # soup = BeautifulSoup(res.text, "html.parser")
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    main_content = soup.find(name="section", attrs={"class": "mainContent__b8b5cf5e"})
    for news in main_content.find_all(name="div", attrs={"class": "storyItem__192ee8af"}):
        category = news.find(name="div", attrs={"class": "eyebrow__4b7f0542"}).text
        title = news.find(name="a", attrs={"class": "headline__55bd5397"}).text
        date = news.find(name="div", attrs={"class": "publishedAt__79f8aaad"}).text
        url = news.find(name="a", attrs={"class": "headline__55bd5397"}).get("href")
        yield [category, title, date, url]

In [63]:
class BloombergCrawler:
    
    def __init__(self):
        self.driver_path = r"./chromedriver.exe"
        self.load_more_result = '//*[@id="root"]/div/section[2]/div[3]/div[12]/button'
        
    def parse(self, query, time_difference=30):
        opts = Options()
        opts.add_argument("--incognito")
        opts.add_argument("user-agent={}".format(random.choice(USER_AGENTS)))
        start_time = time.time()
        url = "https://www.bloomberg.com/search?query={}".format(query)
        driver = webdriver.Chrome(self.driver_path, options=opts)
        driver.get(url)
        time.sleep(random.randint(3, 5))
        self.scroll_to_bottom(driver, time_difference=30)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        driver.quit()
        
        generate_news = self.news_generator(soup)
        news_list = list(generate_news)
        data = pd.DataFrame(news_list, columns=["category", "title", "date", "url"])
        print("Elapsed time: {}s".format(time.time() - start_time))
        return data
        
    def check_exists_by_xpath(self, driver, xpath):
        try:
            driver.find_element_by_xpath(xpath)
        except NoSuchElementException:
            return False
        return True

    def scroll_to_bottom(self, driver, time_difference=30):
        start_time = time.time()
        old_position = 0
        new_position = None

        while new_position != old_position:
            # Get old scroll position
            old_position = driver.execute_script(
                    ("return (window.pageYOffset !== undefined) ?"
                     " window.pageYOffset : (document.documentElement ||"
                     " document.body.parentNode || document.body);"))
            # Sleep and Scroll
            time.sleep(1)
            driver.execute_script((
                    "var scrollingElement = (document.scrollingElement ||"
                    " document.body);scrollingElement.scrollTop ="
                    " scrollingElement.scrollHeight;"))
            time.sleep(random.randint(3, 5))
            if self.check_exists_by_xpath(driver, self.load_more_result):
                driver.find_element_by_xpath(self.load_more_result).click()
            time.sleep(random.randint(3, 5))
            # Get new position
            new_position = driver.execute_script(
                    ("return (window.pageYOffset !== undefined) ?"
                     " window.pageYOffset : (document.documentElement ||"
                     " document.body.parentNode || document.body);"))
            if time.time() - start_time >= time_difference:
                break
            
    def news_generator(self, soup):
        main_content = soup.find(name="section", attrs={"class": "mainContent__b8b5cf5e"})
        for news in main_content.find_all(name="div", attrs={"class": "storyItem__192ee8af"}):
            category = news.find(name="div", attrs={"class": "eyebrow__4b7f0542"}).text
            title = news.find(name="a", attrs={"class": "headline__55bd5397"}).text
            date = news.find(name="div", attrs={"class": "publishedAt__79f8aaad"}).text
            url = news.find(name="a", attrs={"class": "headline__55bd5397"}).get("href")
            yield [category, title, date, url]

In [69]:
crawler = BloombergCrawler()
data = crawler.parse(query="GOOG")

WebDriverException: Message: chrome not reachable
  (Session info: chrome=84.0.4147.105)


In [67]:
data

,category,title,date,url
0,technology,Apple’s Marketing Head Phil Schiller Replaced ...,"August 4, 2020",https://www.bloomberg.com/news/articles/2020-0...
1,technology,"As Apple Nears $2 Trillion, Its Share of S&P 5...","August 4, 2020",https://www.bloomberg.com/news/articles/2020-0...
2,opinion,$5 Trillion Data Hogs Amazon Apple Facebook an...,"August 4, 2020",https://www.bloomberg.com/view/articles/2020-0...
3,opinion,"Apple, Amazon, Facebook and Google Faithful Fa...","August 3, 2020",https://www.bloomberg.com/view/articles/2020-0...
4,technology,Apple Purges Thousands of Unlicensed Games fro...,"August 2, 2020",https://www.bloomberg.com/news/articles/2020-0...
5,markets,Why Investors Keep Losing Money Betting Agains...,"August 3, 2020",https://www.bloomberg.com/news/articles/2020-0...
6,business,Apple Vendor Looks to Shift Production Lines t...,"August 1, 2020",https://www.bloomberg.com/news/articles/2020-0...
7,business,Apple Asks U.K. Store Landlords to Halve Rent:...,"August 2, 2020",https://www.bloomberg.com/news/articles/2020-0...
8,business,Fortnite Owner Says Apple Supports Unfair Busi...,"July 31, 2020",https://www.bloomberg.com/news/videos/2020-07-...
9,business,"Samsung, Apple to Boost Cellphone Manufacturin...","August 1, 2020",https://www.bloomberg.com/news/articles/2020-0...
